Goal: Build and evaluate machine learning models to classify customers into risk tiers (Low / Medium / High) in a realistic FinTech compliance context. Compare linear and tree-based models, evaluate performance using business relevant metrics, and interpret results for compliance prioritization.

- Run this notebook top-to-bottom.
- Data path: `../data/fintech_synthetic_5000.csv`
- Environment: see `requirements.txt`

In [1]:
import pandas as pd
import numpy as np

#Load data
df = pd.read_csv("../data/fintech_synthetic_5000.csv")
df.columns = df.columns.str.strip()

In [2]:
#Train/Test Split
from sklearn.model_selection import train_test_split

y = df["risk_tier"]
X = df.drop(columns=["customer_id", "risk_tier", "next_30d_net_revenue_eur"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

y_train.value_counts(normalize=True)


risk_tier
Low       0.526667
Medium    0.365067
High      0.108267
Name: proportion, dtype: float64

In [3]:
#Building preprocessing pipeline

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_features = X.select_dtypes(include=["object"]).columns.tolist()
numerical_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

numeric_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, numerical_features),
        ("cat", categorical_pipeline, categorical_features)
    ]
)


In [4]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

clf_lr = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", LogisticRegression(max_iter=2000))
])

clf_lr.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers co

In [5]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier

clf_rf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestClassifier(
        n_estimators=300,
        random_state=42,
        class_weight="balanced"
    ))
])

clf_rf.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers co

In [6]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score

def evaluate_classifier(model, X_test, y_test, name):
    preds = model.predict(X_test)
    f1 = f1_score(y_test, preds, average="macro")
    print(f"===== {name} =====")
    print(f"F1-macro: {f1:.4f}\n")
    print("Confusion matrix (rows=true, cols=pred):")
    print(confusion_matrix(y_test, preds, labels=["Low", "Medium", "High"]))
    print("\nClassification report:")
    print(classification_report(y_test, preds))

evaluate_classifier(clf_lr, X_test, y_test, "Logistic Regression")
evaluate_classifier(clf_rf, X_test, y_test, "Random Forest")


===== Logistic Regression =====
F1-macro: 0.6783

Confusion matrix (rows=true, cols=pred):
[[549 108   1]
 [117 312  27]
 [  0  74  62]]

Classification report:
              precision    recall  f1-score   support

        High       0.69      0.46      0.55       136
         Low       0.82      0.83      0.83       658
      Medium       0.63      0.68      0.66       456

    accuracy                           0.74      1250
   macro avg       0.71      0.66      0.68      1250
weighted avg       0.74      0.74      0.74      1250

===== Random Forest =====
F1-macro: 0.7191

Confusion matrix (rows=true, cols=pred):
[[563  95   0]
 [102 344  10]
 [  0  74  62]]

Classification report:
              precision    recall  f1-score   support

        High       0.86      0.46      0.60       136
         Low       0.85      0.86      0.85       658
      Medium       0.67      0.75      0.71       456

    accuracy                           0.78      1250
   macro avg       0.79      0.

In [7]:
#Random Forest was selected as the primary classification model due to its higher macro F1 score and significantly better precision for the High-risk class,
#reducing false positives in compliance workflows while maintaining conservative risk detection.


In [8]:
def get_feature_names(preprocessor):
    num_features = numerical_features
    
    cat_features = preprocessor.named_transformers_["cat"] \
        .named_steps["encoder"] \
        .get_feature_names_out(categorical_features)
    
    return np.concatenate([num_features, cat_features])


In [9]:
import pandas as pd

# Access trained Random Forest inside pipeline
rf_model = clf_rf.named_steps["model"]
feature_names = get_feature_names(clf_rf.named_steps["preprocessor"])

importances = pd.Series(
    rf_model.feature_importances_,
    index=feature_names
).sort_values(ascending=False)

importances.head(15)


aml_alerts_180d        0.170916
chargeback_cnt_90d     0.123649
failed_txn_rate        0.093699
disputes_open          0.054162
cross_border_ratio     0.053243
avg_txn_amount_eur     0.047438
cash_in_ratio          0.047366
account_age_days       0.046708
income_annual_eur      0.046240
avg_session_min        0.045188
age                    0.041114
txn_cnt_30d            0.036829
logins_30d             0.032699
support_tickets_90d    0.031247
pep_flag               0.011280
dtype: float64

In [10]:
aggregated_importance = (
    importances
    .rename("importance")
    .reset_index()
    .assign(base_feature=lambda x: x["index"].str.split("_").str[0])
    .groupby("base_feature", as_index=False)["importance"]
    .sum()
    .sort_values("importance", ascending=False)
)

aggregated_importance.head(10)


,base_feature,importance
2,aml,0.170916
6,chargeback,0.123649
12,failed,0.093699
3,avg,0.092626
11,disputes,0.054162
8,cross,0.053243
5,cash,0.047366
0,account,0.046708
14,income,0.046240
1,age,0.041114


In [11]:
#The model identifies AML alerts, chargebacks, and transaction failures as the primary drivers of customer risk, followed by cross-border activity and dispute behavior. 
#Customer stability indicators such as account age and engagement reduce predicted risk, while demographic attributes play a limited role. 
#This aligns with a risk-based compliance approach and supports prioritization of high-risk customers without excessive false positives.
